In [1]:
from time import time

start_time = time()

for _ in range(1000000):
    pass

time_taken = (time() - start_time) * 1000

print('Executed in %f ms' % time_taken)

assert(time_taken > 0)

Executed in 50.406933 ms


In [2]:
context = {}

context['symbols'] = [
    'BP',
    'CD',
    'CL',
    'ED',
    'GC',
    'HG',
    'HO',
    'HU',
    'JY',
    'SB',
    'SF',
    'SP',
    'SV',
    'TB',
    'TY',
    'US'
]

assert(len(context['symbols']) == 16)

In [3]:
from quantopian.research.experimental import continuous_future

context['markets'] = map(
    lambda market: continuous_future(market),
    context['symbols']
)

assert(len(context['markets']) == 16)

In [4]:
from quantopian.research.experimental import history

fields = ['high', 'low', 'close_price']
start_date = '2017-02-27'
end_date = '2017-03-31'
frequency = 'daily'

context['prices'] = history(
    context['markets'],
    fields=fields,
    frequency=frequency,
    start_date=start_date,
    end_date=end_date
)

assert(context['prices'].shape == (3, 25, 16))

In [5]:
context['prices'].dropna(axis=2, inplace=True)

assert(context['prices'].shape == (3, 25, 14))

In [6]:
if len(context['prices'].items) == len(fields):
    context['prices'] = context['prices'].transpose(2, 1, 0)
    context['prices'] = context['prices'].reindex()

assert(context['prices'].shape == (14, 25, 3))

In [7]:
from talib import ATR

context['average_true_range'] = {}

rolling_window = 21
moving_average = 20

for market in context['prices'].items:
    context['average_true_range'][market] = ATR(
        context['prices'][market].high[-rolling_window:],
        context['prices'][market].low[-rolling_window:],
        context['prices'][market].close_price[-rolling_window:],
        timeperiod=moving_average
    )[-1]

assert(len(context['average_true_range']) == 14)

In [8]:
fields = ['contract']
start_date = '2017-03-31'
end_date = '2017-03-31'

context['contracts'] = history(
    context['markets'],
    fields=fields,
    frequency=frequency,
    start_date=start_date,
    end_date=end_date
)

assert(context['contracts'].shape == (1, 1, 16))

In [9]:
if len(context['contracts'].items) == len(fields):
    context['contracts'] = context['contracts'].transpose(2, 1, 0)
    context['contracts'] = context['contracts'].reindex()

assert(context['contracts'].shape == (16, 1, 1))

In [10]:
for market in context['contracts']:
    contract = context['contracts'][market].contract[0]
    
    if contract is not None:
        print(
            contract.symbol,
            contract.tick_size,
            contract.multiplier,
            contract.auto_close_date
        )
        
        if contract.symbol == 'USM17':
            assert(contract.tick_size == 0.03125)

(u'BPM17', 0.0001, 62500.0, Timestamp('2017-06-15 00:00:00+0000', tz='UTC'))
(u'CDM17', 0.0001, 100000.0, Timestamp('2017-06-16 00:00:00+0000', tz='UTC'))
(u'CLK17', 0.01, 1000.0, Timestamp('2017-04-18 00:00:00+0000', tz='UTC'))
(u'EDJ17', 0.005, 2500.0, Timestamp('2017-04-12 00:00:00+0000', tz='UTC'))
(u'GCM17', 0.1, 100.0, Timestamp('2017-05-29 00:00:00+0000', tz='UTC'))
(u'HGK17', 0.0005, 25000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'HOK17', 0.0001, 42000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'JYM17', 5e-07, 12500000.0, Timestamp('2017-06-15 00:00:00+0000', tz='UTC'))
(u'SBK17', 0.0001, 112000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'SFM17', 0.0001, 125000.0, Timestamp('2017-06-15 00:00:00+0000', tz='UTC'))
(u'SPM17', 0.1, 250.0, Timestamp('2017-06-13 00:00:00+0000', tz='UTC'))
(u'SVK17', 0.005, 5000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'TYM17', 0.015625, 1000.0, Timestamp('2017-05-29 00:00:00+0000', tz='UTC'))
(u'USM17',

In [11]:
for market in context['contracts']:
    contract = context['contracts'][market].contract[0]
    
    if contract is not None:
        dollar_volatility = contract.multiplier * context['average_true_range'][market]
        
        print(market.root_symbol, dollar_volatility)
        
        if market.root_symbol == 'HO':
            assert(dollar_volatility == 1423.7999999999979)

('BP', 640.62499999999739)
('CD', 430.00000000000034)
('CL', 1244.1000000000008)
('ED', 51.249999999997797)
('GC', 1224.3899999999962)
('HG', 1207.5000000000002)
('HO', 1423.7999999999979)
('JY', 0.0)
('SB', 582.39999999999986)
('SF', 818.75000000000421)
('SP', 3884.8124999999809)
('SV', 1315.0000000000048)
('TY', 487.39999999999884)
('US', 1188.9500000000025)


In [12]:
context['capital_risk_per_trade'] = 0.01
context['starting_cash'] = 1000000
# context['portfolio_value'] = 1100000
context['portfolio_value'] = 900000
context['capital_multiplier'] = 2

profit = context['portfolio_value'] - context['starting_cash']
context['cash'] = context['starting_cash']

if profit < 0:
    context['cash'] = context['starting_cash'] + profit * context['capital_multiplier']

capital_per_trade = context['cash'] * context['capital_risk_per_trade']
trade_size = capital_per_trade / dollar_volatility

print('Trade size: %i' % int(trade_size))

assert(int(trade_size) == 6)

Trade size: 6


In [13]:
context['twenty_day_breakout'] = 20
context['fifty_five_day_breakout'] = 55

In [14]:
for market in context['prices'].items:
    print(market.root_symbol, context['prices'][market].close_price[-1])
    
    if market.root_symbol == 'SF':
        assert(context['prices'][market].close_price[-1] == 1.002)

('BP', 1.256)
('CD', 0.752)
('CL', 50.850000000000001)
('ED', 98.709999999999994)
('GC', 1251.5999999999999)
('HG', 2.6560000000000001)
('HO', 1.577)
('JY', 0.0089999999999999993)
('SB', 0.16800000000000001)
('SF', 1.002)
('SP', 2358.4000000000001)
('SV', 18.274999999999999)
('TY', 124.625)
('US', 150.96899999999999)


In [15]:
context['twenty_day_high'] = {}
context['twenty_day_low'] = {}

for market in context['prices'].items:
    context['twenty_day_high'][market] = context['prices'][market]\
        .high[-context['twenty_day_breakout']-1:-1].max()
        
    context['twenty_day_low'][market] = context['prices'][market]\
        .low[-context['twenty_day_breakout']-1:-1].min()
    
    if market.root_symbol == 'BP':
        assert(context['prices'][market].high[-context['twenty_day_breakout']-1:-1].min() == 1.222)

In [16]:
context['fifty_five_day_high'] = {}
context['fifty_five_day_low'] = {}

for market in context['prices'].items:
    context['fifty_five_day_high'][market] = context['prices'][market]\
        .high[-context['fifty_five_day_breakout']-1:-1].max()
        
    context['fifty_five_day_low'][market] = context['prices'][market]\
        .low[-context['fifty_five_day_breakout']-1:-1].min()
    
    if market.root_symbol == 'GC':
        assert(context['prices'][market].high[-context['fifty_five_day_breakout']-1:-1].max() == 1268.337)